In [14]:
%%capture
!pip install chromadb
!pip install datasets
!pip install pyarrow
!pip install open-clip-torch
!pip install sentence-transformers
!pip install langchain_core langchain_openai

In [15]:
import IPython
from IPython.display import HTML, display, Image, Markdown, Video, Audio
from typing import Optional, Sequence, List, Dict, Union

import soundfile as sf

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from google.colab import userdata

from sentence_transformers import SentenceTransformer
from transformers import ClapModel, ClapProcessor
from datasets import load_dataset

import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from chromadb.api.types import Document, Embedding, EmbeddingFunction, URI, DataLoader

import numpy as np
import torchaudio
import base64
import torch
import json
import cv2
import os

In [16]:
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [17]:
path= "mm_vdb"
client= chromadb.PersistentClient(path= path)

In [18]:
%%capture
!unzip StockVideos-CC0.zip

In [19]:
def extract_frames(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for video_filename in os.listdir(video_folder):
        if video_filename.endswith('.mp4'):
            video_path = os.path.join(video_folder, video_filename)
            video_capture = cv2.VideoCapture(video_path)
            fps = video_capture.get(cv2.CAP_PROP_FPS)
            frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps

            output_subfolder = os.path.join(output_folder, os.path.splitext(video_filename)[0])
            if not os.path.exists(output_subfolder):
                os.makedirs(output_subfolder)

            success, image = video_capture.read()
            frame_number = 0
            while success:
                if frame_number == 0 or frame_number % int(fps * 5) == 0 or frame_number == frame_count - 1:
                    frame_time = frame_number / fps
                    output_frame_filename = os.path.join(output_subfolder, f'frame_{int(frame_time)}.jpg')
                    cv2.imwrite(output_frame_filename, image)

                success, image = video_capture.read()
                frame_number += 1

            video_capture.release()

video_folder_path = 'StockVideos-CC0'
output_folder_path = 'StockVideos-CC0-frames'

extract_frames(video_folder_path, output_folder_path)    

FileNotFoundError: [Errno 2] No such file or directory: 'StockVideos-CC0'

In [ ]:
video_collection = client.get_or_create_collection(
    name='video_collection',
    embedding_function=CLIP,
    data_loader=image_loader
)

In [ ]:
def add_frames_to_chromadb(video_dir, frames_dir):
    video_frames = {}

    for video_file in os.listdir(video_dir):
        if video_file.endswith('.mp4'):
            video_title = video_file[:-4]
            frame_folder = os.path.join(frames_dir, video_title)
            if os.path.exists(frame_folder):
                # List all jpg files in the folder
                video_frames[video_title] = [f for f in os.listdir(frame_folder) if f.endswith('.jpg')]

    ids = []
    uris = []
    metadatas = []

    for video_title, frames in video_frames.items():
        video_path = os.path.join(video_dir, f"{video_title}.mp4")
        for frame in frames:
            frame_id = f"{frame[:-4]}_{video_title}"
            frame_path = os.path.join(frames_dir, video_title, frame)
            ids.append(frame_id)
            uris.append(frame_path)
            metadatas.append({'video_uri': video_path})

    video_collection.add(ids=ids, uris=uris, metadatas=metadatas)

video_dir = 'StockVideos-CC0'
frames_dir = 'StockVideos-CC0-frames'

add_frames_to_chromadb(video_dir, frames_dir)

In [ ]:
def display_videos(query_text, max_distance=None, max_results=5, debug=False):
    displayed_videos = set()

    results = video_collection.query(
        query_texts=[query_text],
        n_results=max_results,  # Adjust the number of results if needed
        include=['uris', 'distances', 'metadatas']
    )

    uris = results['uris'][0]
    distances = results['distances'][0]
    metadatas = results['metadatas'][0]

    for uri, distance, metadata in zip(uris, distances, metadatas):
        video_uri = metadata['video_uri']

        if (max_distance is None or distance <= max_distance) and video_uri not in displayed_videos:
            if debug:
              print(f"URI: {uri} - Video URI: {video_uri} - Distance: {distance}")
            display(Video(video_uri, embed=True, width=300))
            displayed_videos.add(video_uri)  # Add to the set to prevent duplication
        else:
            if debug:
              print(f"URI: {uri} - Video URI: {video_uri} - Distance: {distance} (Filtered out)")


display_videos("trees", max_distance=1.55, debug=True)